In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

r = requests.get("https://raw.githubusercontent.com/eyucao/csvhold/master/Japan_population_data.csv")
df = BeautifulSoup(r.text)

C:\Users\Eric\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Eric\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


I started my cleaning here. Dropping all the NaN/empty values was my first priority. Then I renamed my columns but I noticed a '\r' escape character present in both my column names as well as the island column. To rectify the column name, I simply set the sixth entry(island), equal to itself but with the last character removed. I also dropped the last row as it contained all None values that dropna didn't pick up when I tried this before.

I then calculated the population change in each prefecture. I did this by calculating the previous row's population and simply looking at the delta. The issue with using the widely used method of shift is that our data is dependent on the prefecture. So for us to get relevant population change per year per prefecture, we have to do it manually. It looks ugly but it does what it needs to do.

After creating a dummy column for this new feature, I run through the dictionary I've created that has mapped indices to the population change per year. I then run through this dictionary in a for loop and utilize the .iat method to place these datapoints in their correct respective locations.

Now we can use shift to see the change in population change per year per prefecture(population delta delta). After doing our shift, both our population change change and percent population delta delta are off by a position of one and have extraneous entries in the intersection between features(remember the problem we had earlier with not being able to use shift).

This situation is different though! We can do our shifting, and then simply go through and set the rows where a new prefecture begins values of population delta delta and percent delta delta to zero. Done!

Next we visualize our data in hopes of confirming or denying our hypothesis that the population growth decline in Japan is region-agnostic. We are met with that conclusion that it is due to all regions expressing similar trends in population delta delta. Due to this information, we can simply use national statistics instead.

Now that we know this, we sum up all yearly totals across prefectures per year to get a cumulative sum per year. Index 14 is double counted due to a bug, so we just rectify the value by dividing it in two. After we do this, we now are left with a year by year population table. However, this data has little use as we are more interested in both the population delta and population delta delta(and the percent change). We use our good friend shift once again and end up where we want.

At this point, we bring in relevant data generally associated with population decline(fertility and mortality as per the Population Reference Bureau's recommendation) as well as many that are not conventionally used. We now need to establish that these datasets present a somewhat linear relationship to a decline in population change.

Next we run our data through multiple regression and score it to see how accurately it can depict this relationship. We split the data into training(x) and results(y). We then split these two dataframes 90:10 training and scoring, respectively. After we reach our score, we wonder if a different method might yield an even higher score. 

This is where we learned about Gradient Descent Boosting. To our understanding, gradient boosting works using improvement on three factors. These factors being optimizing a loss function, using decision trees as greedy weak learner, and an additive model that utilizes these weak learners in an attempt to minimize the loss.

This is where it gets really interesting! We now use linear regression on each of the relevant features that have an impact on our population decline. By doing this we aim to project the population decline in 2025(this is important for later) as well as the size of the workforce and the general population size by this year.

In [2]:
df2 = df.getText().split('\n')
i = 0
df3 = []
for line in df2:
    if i == 0:
        colnames = line.split(',')
        i = 1
    else:
        df3.append(line.split(','))
df3 = df3[:-1]
colnames[6] = colnames[6][:-1]
population_data = pd.DataFrame(data = df3, columns = colnames)
population_data.dropna()
population_data.reset_index(drop=True, inplace = True)
population_data.head(10)

,prefecture,year,population,capital,region,estimated_area,island
0,Aichi-ken,1872.1667,1210368,Nagoya-shi,Chubu,5165.12,Honshu\r
1,Aichi-ken,1873,1217444,Nagoya-shi,Chubu,5165.12,Honshu\r
2,Aichi-ken,1874,1217521,Nagoya-shi,Chubu,5165.12,Honshu\r
3,Aichi-ken,1875,1234003,Nagoya-shi,Chubu,5165.12,Honshu\r
4,Aichi-ken,1876,1244711,Nagoya-shi,Chubu,5165.12,Honshu\r
5,Aichi-ken,1877,1250839,Nagoya-shi,Chubu,5165.12,Honshu\r
6,Aichi-ken,1878,1267206,Nagoya-shi,Chubu,5165.12,Honshu\r
7,Aichi-ken,1879,1295452,Nagoya-shi,Chubu,5165.12,Honshu\r
8,Aichi-ken,1880,1303812,Nagoya-shi,Chubu,5165.12,Honshu\r
9,Aichi-ken,1881,1317792,Nagoya-shi,Chubu,5165.12,Honshu\r


In [3]:
for index, row in population_data.iterrows():
    row['island'] = str(row['island'])[:-1]
    # print(row['island'])
population_data.head(10)

,prefecture,year,population,capital,region,estimated_area,island
0,Aichi-ken,1872.1667,1210368,Nagoya-shi,Chubu,5165.12,Honshu
1,Aichi-ken,1873,1217444,Nagoya-shi,Chubu,5165.12,Honshu
2,Aichi-ken,1874,1217521,Nagoya-shi,Chubu,5165.12,Honshu
3,Aichi-ken,1875,1234003,Nagoya-shi,Chubu,5165.12,Honshu
4,Aichi-ken,1876,1244711,Nagoya-shi,Chubu,5165.12,Honshu
5,Aichi-ken,1877,1250839,Nagoya-shi,Chubu,5165.12,Honshu
6,Aichi-ken,1878,1267206,Nagoya-shi,Chubu,5165.12,Honshu
7,Aichi-ken,1879,1295452,Nagoya-shi,Chubu,5165.12,Honshu
8,Aichi-ken,1880,1303812,Nagoya-shi,Chubu,5165.12,Honshu
9,Aichi-ken,1881,1317792,Nagoya-shi,Chubu,5165.12,Honshu


In [4]:
for index, row in population_data.iterrows():
    #print(row['year'])
    row['year'] = int(float(row['year']))
population_data

,prefecture,year,population,capital,region,estimated_area,island
0,Aichi-ken,1872,1210368,Nagoya-shi,Chubu,5165.12,Honshu
1,Aichi-ken,1873,1217444,Nagoya-shi,Chubu,5165.12,Honshu
2,Aichi-ken,1874,1217521,Nagoya-shi,Chubu,5165.12,Honshu
3,Aichi-ken,1875,1234003,Nagoya-shi,Chubu,5165.12,Honshu
4,Aichi-ken,1876,1244711,Nagoya-shi,Chubu,5165.12,Honshu
5,Aichi-ken,1877,1250839,Nagoya-shi,Chubu,5165.12,Honshu
6,Aichi-ken,1878,1267206,Nagoya-shi,Chubu,5165.12,Honshu
7,Aichi-ken,1879,1295452,Nagoya-shi,Chubu,5165.12,Honshu
8,Aichi-ken,1880,1303812,Nagoya-shi,Chubu,5165.12,Honshu
9,Aichi-ken,1881,1317792,Nagoya-shi,Chubu,5165.12,Honshu


In [5]:
prev = {}
popdelta = {}
population_data = population_data[population_data.population != 'NA']
population_data.reset_index(drop=True, inplace = True)
for index, row in population_data.iterrows():
    if row['prefecture'] in prev:
        #print(row['population'])
        popdelta[index] = int(row['population']) - int(prev[row['prefecture']])
        prev[row['prefecture']] = row['population']
    else:
        popdelta[index] = 0
        prev[row['prefecture']] = row['population']
popdelta

,prefecture,year,population,capital,region,estimated_area,island
0,Aichi-ken,1872,1210368,Nagoya-shi,Chubu,5165.12,Honshu
1,Aichi-ken,1873,1217444,Nagoya-shi,Chubu,5165.12,Honshu
2,Aichi-ken,1874,1217521,Nagoya-shi,Chubu,5165.12,Honshu
3,Aichi-ken,1875,1234003,Nagoya-shi,Chubu,5165.12,Honshu
4,Aichi-ken,1876,1244711,Nagoya-shi,Chubu,5165.12,Honshu
5,Aichi-ken,1877,1250839,Nagoya-shi,Chubu,5165.12,Honshu
6,Aichi-ken,1878,1267206,Nagoya-shi,Chubu,5165.12,Honshu
7,Aichi-ken,1879,1295452,Nagoya-shi,Chubu,5165.12,Honshu
8,Aichi-ken,1880,1303812,Nagoya-shi,Chubu,5165.12,Honshu
9,Aichi-ken,1881,1317792,Nagoya-shi,Chubu,5165.12,Honshu


In [6]:
#population_data.append(popdelta)
#ppod = pd.DataFrame(list(range(len(popdelta))))
#pd.concat(ppod,population_data)
#for index, row in ppod.iterrows():
#    row[0] = popdelta[index]
#ppod
population_data['pop_delta'] = 1
for index,row in population_data.iterrows():
    if index in popdelta.keys():
        #row['pop_delta']
        population_data.iat[index,7] = popdelta[index]
population_data

C:\Users\Eric\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,prefecture,year,population,capital,region,estimated_area,island,pop_delta
0,Aichi-ken,1872,1210368,Nagoya-shi,Chubu,5165.12,Honshu,0
1,Aichi-ken,1873,1217444,Nagoya-shi,Chubu,5165.12,Honshu,7076
2,Aichi-ken,1874,1217521,Nagoya-shi,Chubu,5165.12,Honshu,77
3,Aichi-ken,1875,1234003,Nagoya-shi,Chubu,5165.12,Honshu,16482
4,Aichi-ken,1876,1244711,Nagoya-shi,Chubu,5165.12,Honshu,10708
5,Aichi-ken,1877,1250839,Nagoya-shi,Chubu,5165.12,Honshu,6128
6,Aichi-ken,1878,1267206,Nagoya-shi,Chubu,5165.12,Honshu,16367
7,Aichi-ken,1879,1295452,Nagoya-shi,Chubu,5165.12,Honshu,28246
8,Aichi-ken,1880,1303812,Nagoya-shi,Chubu,5165.12,Honshu,8360
9,Aichi-ken,1881,1317792,Nagoya-shi,Chubu,5165.12,Honshu,13980


In [7]:
pop_yr = {}

for index, row in population_data.iterrows():
    if row['year'] in pop_yr:
        #print(row['population'])
        pop_yr[row['year']] += int(row['population'])
    else:
        pop_yr[row['year']] = 0
pop_yr

{1872: 26858564,
 1873: 27840112,
 1874: 29388390,
 1875: 29718092,
 1876: 30590571,
 1877: 33377526,
 1878: 33631370,
 1879: 34473104,
 1880: 34625009,
 1881: 35041202,
 1882: 35368068,
 1883: 35662306,
 1884: 36317069,
 1885: 36601650,
 1886: 75723318,
 1887: 38080660,
 1888: 38661468,
 1889: 39236514,
 1890: 39495736,
 1891: 39784988,
 1892: 40199056,
 1893: 40548556,
 1894: 40901636,
 1895: 41494779,
 1896: 41922513,
 1897: 42385762,
 1898: 43763430,
 1903: 46790694,
 1908: 49855747,
 1913: 53058046,
 1918: 55947200,
 1920: 53873291,
 1925: 57417328,
 1930: 61882592,
 1935: 66391447,
 1940: 69947716,
 1944: 69784110,
 1945: 69140253,
 1946: 70195051,
 1947: 74978571,
 1948: 76990780,
 1950: 80726927,
 1955: 86307385,
 1960: 90095237,
 1965: 94410484,
 1970: 99279008,
 1975: 106016074,
 1980: 110838758,
 1985: 114593751,
 1990: 116920564,
 1995: 118701910,
 2000: 119882543,
 2005: 120513290,
 2010: 120646633,
 2015: 119625953}

In [8]:
total_pop = pd.DataFrame([list(pop_yr.keys()),list(pop_yr.values())])
total_pop = total_pop.T
total_pop.columns = ['year','population']
total_pop.iat[14,1] = int(total_pop.iat[14,1] /2)
total_pop

,year,population
0,1872,26858564
1,1873,27840112
2,1874,29388390
3,1875,29718092
4,1876,30590571
5,1877,33377526
6,1878,33631370
7,1879,34473104
8,1880,34625009
9,1881,35041202


In [9]:
#df['dA'] = df['A'] - df['A'].shift(-1)
total_pop['pop_delta'] = -1 *( total_pop['population'] - total_pop['population'].shift(-1))
total_pop['pop_delta'] = total_pop['pop_delta'].shift(1)
total_pop.head(10)

,year,population,pop_delta
0,1872,26858564,NaN
1,1873,27840112,981548.0
2,1874,29388390,1548278.0
3,1875,29718092,329702.0
4,1876,30590571,872479.0
5,1877,33377526,2786955.0
6,1878,33631370,253844.0
7,1879,34473104,841734.0
8,1880,34625009,151905.0
9,1881,35041202,416193.0


In [10]:
total_pop['pop_delta_delta'] = -1* ( total_pop['pop_delta'] - total_pop['pop_delta'].shift(-1))
total_pop['pop_delta_delta'] = total_pop['pop_delta_delta'].shift(1)
total_pop.head(10)

,year,population,pop_delta,pop_delta_delta
0,1872,26858564,NaN,NaN
1,1873,27840112,981548.0,NaN
2,1874,29388390,1548278.0,566730.0
3,1875,29718092,329702.0,-1218576.0
4,1876,30590571,872479.0,542777.0
5,1877,33377526,2786955.0,1914476.0
6,1878,33631370,253844.0,-2533111.0
7,1879,34473104,841734.0,587890.0
8,1880,34625009,151905.0,-689829.0
9,1881,35041202,416193.0,264288.0


In [11]:
total_pop['pd_decrease'] = -1 * ( total_pop['pop_delta_delta'].shift(-1) / total_pop['pop_delta'])
#total_pop['pop_delta_delta'] = total_pop['pop_delta_delta'].shift(1)
total_pop['pd_decrease'] = total_pop['pd_decrease'].shift(1)
total_pop

,year,population,pop_delta,pop_delta_delta,pd_decrease
0,1872,26858564,NaN,NaN,NaN
1,1873,27840112,981548.0,NaN,NaN
2,1874,29388390,1548278.0,566730.0,-0.577384
3,1875,29718092,329702.0,-1218576.0,0.787052
4,1876,30590571,872479.0,542777.0,-1.646265
5,1877,33377526,2786955.0,1914476.0,-2.194295
6,1878,33631370,253844.0,-2533111.0,0.908917
7,1879,34473104,841734.0,587890.0,-2.315950
8,1880,34625009,151905.0,-689829.0,0.819533
9,1881,35041202,416193.0,264288.0,-1.739824


In [12]:
pref = {}
for index, row in population_data.iterrows():
    if row['prefecture'] not in pref.keys():
        pref[row['prefecture']] = 0
#need to get delta delta
population_data['pop_delta_delta'] = -1 *(population_data['pop_delta'] - population_data['pop_delta'].shift(-1))
population_data['pop_delta_delta'] = population_data['pop_delta_delta'].shift(1)

population_data['pdd_percent'] = -1 *(population_data['pop_delta_delta'].shift(-1) / population_data['pop_delta'])
population_data['pdd_percent'] = population_data['pdd_percent'].shift(1)
population_data

C:\Users\Eric\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Eric\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Eric\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

,prefecture,year,population,capital,region,estimated_area,island,pop_delta,pop_delta_delta,pdd_percent
0,Aichi-ken,1872,1210368,Nagoya-shi,Chubu,5165.12,Honshu,0,NaN,NaN
1,Aichi-ken,1873,1217444,Nagoya-shi,Chubu,5165.12,Honshu,7076,7076.0,-inf
2,Aichi-ken,1874,1217521,Nagoya-shi,Chubu,5165.12,Honshu,77,-6999.0,0.989118
3,Aichi-ken,1875,1234003,Nagoya-shi,Chubu,5165.12,Honshu,16482,16405.0,-213.051948
4,Aichi-ken,1876,1244711,Nagoya-shi,Chubu,5165.12,Honshu,10708,-5774.0,0.350322
5,Aichi-ken,1877,1250839,Nagoya-shi,Chubu,5165.12,Honshu,6128,-4580.0,0.427718
6,Aichi-ken,1878,1267206,Nagoya-shi,Chubu,5165.12,Honshu,16367,10239.0,-1.670855
7,Aichi-ken,1879,1295452,Nagoya-shi,Chubu,5165.12,Honshu,28246,11879.0,-0.725790
8,Aichi-ken,1880,1303812,Nagoya-shi,Chubu,5165.12,Honshu,8360,-19886.0,0.704029
9,Aichi-ken,1881,1317792,Nagoya-shi,Chubu,5165.12,Honshu,13980,5620.0,-0.672249


In [14]:
prevs = []

for index, row in population_data.iterrows():
    #print(row['prefecture'])
    if row['prefecture'] not in prevs:
        prevs.append(row['prefecture'])
        population_data.iat[index,9] = 0
        population_data.iat[index,8] = 0
#print(prevs)

population_data['pop_delta_delta'] = population_data['pop_delta_delta'].shift(-1)
population_data['pdd_percent'] = population_data['pdd_percent'].shift(-1)
prevs.clear()
for index, row in population_data.iterrows():
    #print(row['prefecture'])
    if row['prefecture'] not in prevs:
        prevs.append(row['prefecture'])
        population_data.iat[index,9] = 0
        population_data.iat[index,8] = 0

population_data['pop_delta_delta'] = population_data['pop_delta_delta'].shift(1)
population_data['pdd_percent'] = population_data['pdd_percent'].shift(1)
population_data

C:\Users\Eric\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Eric\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\Eric\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

,prefecture,year,population,capital,region,estimated_area,island,pop_delta,pop_delta_delta,pdd_percent
0,Aichi-ken,1872,1210368,Nagoya-shi,Chubu,5165.12,Honshu,0,NaN,NaN
1,Aichi-ken,1873,1217444,Nagoya-shi,Chubu,5165.12,Honshu,7076,0.0,0.000000
2,Aichi-ken,1874,1217521,Nagoya-shi,Chubu,5165.12,Honshu,77,-6999.0,0.989118
3,Aichi-ken,1875,1234003,Nagoya-shi,Chubu,5165.12,Honshu,16482,16405.0,-213.051948
4,Aichi-ken,1876,1244711,Nagoya-shi,Chubu,5165.12,Honshu,10708,-5774.0,0.350322
5,Aichi-ken,1877,1250839,Nagoya-shi,Chubu,5165.12,Honshu,6128,-4580.0,0.427718
6,Aichi-ken,1878,1267206,Nagoya-shi,Chubu,5165.12,Honshu,16367,10239.0,-1.670855
7,Aichi-ken,1879,1295452,Nagoya-shi,Chubu,5165.12,Honshu,28246,11879.0,-0.725790
8,Aichi-ken,1880,1303812,Nagoya-shi,Chubu,5165.12,Honshu,8360,-19886.0,0.704029
9,Aichi-ken,1881,1317792,Nagoya-shi,Chubu,5165.12,Honshu,13980,5620.0,-0.672249


In [17]:
print(type(total_pop.iat[0,0]))

<class 'numpy.int64'>


In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split

total_pop.drop(total_pop.index[:2], inplace=True)

reg = LinearRegression()

labels = total_pop['population']
train1 = total_pop.drop(['population'],axis=1)

x_train , x_test , y_train , y_test = train_test_split(train1 , labels , test_size = 0.10,random_state =2)

reg.fit(x_train,y_train)
reg.score(x_test,y_test)

0.9673392181677011

In [21]:
from sklearn import ensemble
clf = ensemble.GradientBoostingRegressor(n_estimators = 400, max_depth = 5, min_samples_split = 2,
          learning_rate = 0.1, loss = 'ls')

clf.fit(x_train, y_train)
clf.score(x_test,y_test)

0.9569775713312038

### Enter the Hinoe Uma(Fire Horse)

Japanese society is extremely homogenous with 99.5% of the population in Japan being ethnically Japanese. As such, this ethnic homogeneity lends itself to widespread beliefs about topics such as work culture, but also in this case superstition. Hinoe Uma, or Fire Horse, is the 43rd combination of the sexagenary cycle. It is believed that children born this year have bad personalities and so many people will actively avoid giving birth during this year. As such, birthrates for this year fall dramatically every time it comes around. This is due to the average childbearing couple in Japan having only one child, and as such want to avoid all risks.

The reason that Hinoe Uma is important, is that the Japanese population and workforce is already shrinking at an incredible rate. If our projections are mostly accurate at the year 2025, it already looks bleak for Japanese society. This is where Hinoe Uma comes in. Hinoe Uma comes in 60 year cycles. The last time it occurred in 1966, causing a roughly 25.9% decline in birthrates for that year. In the year after, it recovered, but the Japanese birthrate was incredibly high at this time due to still being in the process of industrialization. 2026 is the next time Hinoe Uma will occur, without a doubt causing another sharp decline in birthrates. However, this time Japanese birthrates are already at a historical low with no sign of changing in sight. A dramatic decrease in birthrates during the midst of a decline means that Japan misses out on a crucial amount of new children that will likely not be recovered due to the decline that's projected to continue year after year. Hinoe Uma also contribute to a permanent decrease in birthrates for the years after, although causes a high spike the year or two after due to compensation. This is seen as the year immediatedely before 1966 had a high amount of growth but after 1970, the population decline fell under 2.00, never to recover in the 48 years since.

It is unlikely that Hinoe Uma singehandedly kicked off population decline for Japan at large. However, with the rapid post-war industrialization and societal shifts regarding the value of material culture, Japan's population was already incredibly vulnerable. A cultural phenoma that caused widespread birthrate decline may have been the spark that lit the fuse on Japan's already shaky views on the importance of childbirth.

It's unreasonable to try to impute the data for 2065, but if for some reasons all factors remain the same and Japanese society still is victim to population decline, Hinoe Uma will return in 2086 to cause more havoc to an already incredibly low population growth.

### Conclusion

When we chose this topic, we went into it with a curiousity about this extremely interesting sociological problem. By utilizing the techniques we learned in class, we were able to gain new insight to this problem that many sources online are unaware of. We hypothesized that Japanese population decline was more than just the difference between deathrate and birthrate, and our hunch turned out to be correct. Of course population change is simply that subtraction, but it doesn't speak to any length of the cultural undercurrents causing this change. Our project aimed to identify these factors and display their effect on a society.

As we continued with the project, our goals became more and more lofty. We created a regression model and refined our predictions by opting for a more accurate gradient descent boosting algorithm. This allowed us to predict population changes with features that we had identified as having a linear relationship with population decline. 

As we pored over the data we had found, we found strange anomalies, most notably in 1966. We had found the cultural phenomena known natively as Hinoe Uma. This phenomena caused dramatic decline in the year it was present, but had it been in any other society, its effects would hardly be notiacble. However, this was Japan and its effects are much more impactful. We calculated the predicted population and decline by 2025, and used the decrease present in 1966 to model what was to come in 2026.

Now, we know that Japanese population decline is more than just those two features. It depends on urban population, age of workforce, and female literacy and achievement, as well as many more features that we weren't able to cover. This all underscored by what is to come in 2026, which will cause Japan to lose millions more of its population in the years following.